In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

### Originals image path

In [2]:
path = Path('database/originals')
files = [file for file in path.iterdir() if file.is_file()]

## Segmentação Otsu

In [3]:
OTSU_path = 'tests/otsu_original'
Path(OTSU_path).mkdir(parents=True, exist_ok=True)

In [4]:
for file in files:
    image = cv2.imread(str(file), 0)
    name = file.name.split('.')[0]

    _, otsu = cv2.threshold(image, 0, 255,
                            cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    
    cv2.imwrite(OTSU_path + '/' + name + '.jpg', otsu)

## Segmentação padrão

In [5]:
SEG_norm = 'tests/segme_original'
Path(SEG_norm).mkdir(parents=True, exist_ok=True)

In [6]:
for file in files:
    image = cv2.imread(str(file), 0)
    name = file.name.split('.')[0]

    _, thresh = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY_INV)
    
    cv2.imwrite(SEG_norm + '/' + name + '.jpg', thresh)

## Segmentação Adaptativa

In [7]:
ADAP_path = 'tests/adap_original'
Path(ADAP_path).mkdir(parents=True, exist_ok=True)

In [8]:
for file in files:
    image = cv2.imread(str(file), 0)
    name = file.name.split('.')[0]

    adap = cv2.adaptiveThreshold(image,255,cv2.ADAPTIVE_THRESH_MEAN_C,\
                                 cv2.THRESH_BINARY_INV,17,3)
    
    cv2.imwrite(ADAP_path + '/' + name + '.jpg', adap)

In [9]:
ADAP_path = 'tests/adap_original_pos_processado'
Path(ADAP_path).mkdir(parents=True, exist_ok=True)

In [10]:
for file in files:
    image = cv2.imread(str(file), 0)
    name = file.name.split('.')[0]

    adap = cv2.adaptiveThreshold(image,255,cv2.ADAPTIVE_THRESH_MEAN_C,\
                                 cv2.THRESH_BINARY_INV,17,3)
    
    kernel = np.ones((11, 11), np.uint8) 
    adap = cv2.dilate(adap, kernel, iterations=1) 
    
    kernel = np.ones((5,5), np.uint8)
    adap = cv2.erode(adap, kernel, iterations=1)

    cnts,_ = cv2.findContours(adap, 1, 2)
    
    cp = image.copy()
    adap_cp = adap.copy()
    
    for cnt in cnts:
        area = cv2.contourArea(cnt)
        
        if area < 500:
            rect = cv2.minAreaRect(cnt)
            box = cv2.boxPoints(rect)
            box = np.int0(box)
            cp = cv2.drawContours(cp, [box], 0, (0,0,255), 2)
            adap_cp = cv2.drawContours(adap_cp, [box],-1,(0,0,0), thickness=-1)
    
    cv2.imwrite(ADAP_path + '/' + name + '.jpg', adap_cp)
#     cv2.imwrite(ADAP_path + '/cp_' + name + '.jpg', cp)